In [1]:
! pip install transformers[torch]
! pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
^C


In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data.dataset import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("interview_data/arxiv_data.csv")

In [ ]:
df

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"
...,...,...,...
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']"
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']"
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']"
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV']


In [ ]:
label_df = pd.read_csv("interview_data/category_taxonomy.csv")

In [ ]:
label_df

,category,code,name
0,Computer Science,cs.AI,Artificial Intelligence
1,Computer Science,cs.AR,Hardware Architecture
2,Computer Science,cs.CC,Computational Complexity
3,Computer Science,cs.CE,"Computational Engineering, Finance, and Science"
4,Computer Science,cs.CG,Computational Geometry
...,...,...,...
149,Physics,physics.optics,Optics
150,Physics,physics.plasm-ph,Plasma Physics
151,Physics,physics.pop-ph,Popular Physics
152,Physics,physics.soc-ph,Physics and Society


In [ ]:
codes = label_df["code"].to_list()

In [ ]:
codes

['cs.AI',
 'cs.AR',
 'cs.CC',
 'cs.CE',
 'cs.CG',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.CY',
 'cs.DB',
 'cs.DC',
 'cs.DL',
 'cs.DM',
 'cs.DS',
 'cs.ET',
 'cs.FL',
 'cs.GL',
 'cs.GR',
 'cs.GT',
 'cs.HC',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.MA',
 'cs.MM',
 'cs.MS',
 'cs.NA',
 'cs.NE',
 'cs.NI',
 'cs.OH',
 'cs.OS',
 'cs.PF',
 'cs.PL',
 'cs.RO',
 'cs.SC',
 'cs.SD',
 'cs.SE',
 'cs.SI',
 'cs.SY',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'eess.SY',
 'math.AC',
 'math.AG',
 'math.AP',
 'math.AT',
 'math.CA',
 'math.CO',
 'math.CT',
 'math.CV',
 'math.DG',
 'math.DS',
 'math.FA',
 'math.GM',
 'math.GN',
 'math.GR',
 'math.GT',
 'math.HO',
 'math.IT',
 'math.KT',
 'math.LO',
 'math.MG',
 'math.MP',
 'math.NA',
 'math.NT',
 'math.OA',
 'math.OC',
 'math.PR',
 'math.QA',
 'math.RA',
 'math.RT',
 'math.SG',
 'math.SP',
 'math.ST',
 'q-bio.BM',
 'q-bio.CB',
 'q-bio.GN',
 'q-bio.MN',
 'q-bio.NC',
 'q-bio.OT',
 'q-bio.PE',
 'q-bio.QM',
 'q-bio.SC',
 'q-bio.TO',
 '

In [ ]:

new_df = df["terms"].str.replace("[",'').str.replace("]","").str.replace("'","").str.replace(" ","").str.get_dummies(",")
df

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"
...,...,...,...
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']"
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']"
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']"
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV']


In [ ]:
final_df = pd.concat([df,new_df], axis=1)

final_df

,titles,summaries,terms,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,astro-ph.SR,cond-mat.dis-nn,...,q-fin.PR,q-fin.RM,q-fin.ST,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def concatenate_title_summary(val1, val2):
    return f"Title: {val1}. Summary: {val2}"

final_df['title_and_summary'] = final_df.apply(lambda row: concatenate_title_summary(row['titles'], row['summaries']), axis=1)

final_df
final_df.to_csv("data_cleaned.csv", index=False)

In [2]:
final_df = pd.read_csv("data_cleaned.csv")
final_df



,titles,summaries,terms,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,astro-ph.SR,cond-mat.dis-nn,...,q-fin.RM,q-fin.ST,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH,title_and_summary
0,Per-sample Prediction Intervals for Extreme Le...,Prediction intervals in supervised Machine Lea...,"['cs.LG', 'stat.ML']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Title: Per-sample Prediction Intervals for Ext...
1,FusiformNet: Extracting Discriminative Facial ...,"Over the last several years, research on facia...",['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Title: FusiformNet: Extracting Discriminative ...
2,Exploiting Convolutional Representations for M...,We test this premise and explore representatio...,['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Title: Exploiting Convolutional Representation...
3,3D Object Detection Using Scale Invariant and ...,3D object detection plays an important role in...,['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Title: 3D Object Detection Using Scale Invaria...
4,Heterogeneous Deep Graph Infomax,Graph representation learning is to learn univ...,"['cs.LG', 'cs.SI', 'stat.ML']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Title: Heterogeneous Deep Graph Infomax. Summa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,DUP-Net: Denoiser and Upsampler Network for 3D...,Neural networks are vulnerable to adversarial ...,['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Title: DUP-Net: Denoiser and Upsampler Network...
4996,Regret Analysis in Deterministic Reinforcement...,We consider Markov Decision Processes (MDPs) w...,"['cs.LG', 'stat.ML']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Title: Regret Analysis in Deterministic Reinfo...
4997,ID-Unet: Iterative Soft and Hard Deformation f...,View synthesis is usually done by an autoencod...,['cs.CV'],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Title: ID-Unet: Iterative Soft and Hard Deform...
4998,Improving Reinforcement Learning Based Image C...,"Recently, Reinforcement Learning (RL) approach...","['cs.CV', 'cs.LG', 'stat.ML']",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Title: Improving Reinforcement Learning Based ...


In [3]:
# Efficiently sample 10,000 rows from final_df for training
if len(final_df) > 5000:
    final_df = final_df.sample(n=5000, random_state=42).reset_index(drop=True)

test_split = 0.2
train_df, test_df = train_test_split(
    final_df,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 4000
Number of rows in test set: 1000


# Approach 1: Assume that every class is a unique class of it's own

In [4]:
# Assume 154 classes. So device a classification model with 154 classes

## Approach 1a: Using only Title

In [5]:
not_chosen_columns = ['titles', 'summaries','terms', "title_and_summary"]

In [6]:
label_columns = [col for col in final_df.columns if col not in not_chosen_columns]

In [7]:
label_columns

['astro-ph.CO',
 'astro-ph.EP',
 'astro-ph.GA',
 'astro-ph.HE',
 'astro-ph.IM',
 'astro-ph.SR',
 'cond-mat.dis-nn',
 'cond-mat.mtrl-sci',
 'cond-mat.soft',
 'cond-mat.stat-mech',
 'cond-mat.str-el',
 'cs.AI',
 'cs.AR',
 'cs.CC',
 'cs.CE',
 'cs.CG',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.CY',
 'cs.DB',
 'cs.DC',
 'cs.DL',
 'cs.DM',
 'cs.DS',
 'cs.ET',
 'cs.FL',
 'cs.GR',
 'cs.GT',
 'cs.HC',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.MA',
 'cs.MM',
 'cs.MS',
 'cs.NA',
 'cs.NE',
 'cs.NI',
 'cs.OS',
 'cs.PF',
 'cs.PL',
 'cs.RO',
 'cs.SC',
 'cs.SD',
 'cs.SE',
 'cs.SI',
 'cs.SY',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'eess.SY',
 'gr-qc',
 'hep-ex',
 'hep-ph',
 'hep-th',
 'math-ph',
 'math.AC',
 'math.AG',
 'math.AP',
 'math.AT',
 'math.CA',
 'math.CO',
 'math.CT',
 'math.CV',
 'math.DG',
 'math.DS',
 'math.FA',
 'math.GR',
 'math.GT',
 'math.HO',
 'math.IT',
 'math.LO',
 'math.MG',
 'math.MP',
 'math.NA',
 'math.NT',
 'math.OA',
 'math.OC',
 'math.PR',
 'math

In [8]:
df_labels_train = train_df[label_columns]
df_labels_test = test_df[label_columns]

In [9]:
df_labels_train["cs.CV"]

4805    1
1344    1
4648    0
1259    0
4928    1
       ..
3271    1
184     1
1187    0
4254    1
4736    1
Name: cs.CV, Length: 4000, dtype: int64

In [10]:
labels_list_train = df_labels_train.values.tolist()
labels_list_test = df_labels_test.values.tolist()

In [11]:
len(labels_list_test), len(labels_list_test[0])

(1000, 135)

In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

train_texts = train_df['titles'].tolist()
train_labels = labels_list_train

eval_texts = test_df['titles'].tolist()
eval_labels = labels_list_test

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_encodings = tokenizer(train_texts, padding="max_length", truncation=True, max_length=512)
eval_encodings = tokenizer(eval_texts, padding="max_length", truncation=True, max_length=512)


class TextClassifierDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float).to(device)
        return item

train_dataset = TextClassifierDataset(train_encodings, train_labels)
eval_dataset = TextClassifierDataset(eval_encodings, eval_labels)

title_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=len(label_columns),
)

title_model = title_model.to(device)

training_arguments = TrainingArguments(
    output_dir=".",
    dataloader_pin_memory=False,
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
)

trainer = Trainer(
    model=title_model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2500 [00:00<?, ?it/s]

{'loss': 0.0846, 'grad_norm': 0.056887611746788025, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.03666967898607254, 'eval_runtime': 8.5976, 'eval_samples_per_second': 116.312, 'eval_steps_per_second': 14.539, 'epoch': 1.0}
{'loss': 0.0317, 'grad_norm': 0.05817371979355812, 'learning_rate': 3e-05, 'epoch': 2.0}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.028210116550326347, 'eval_runtime': 8.6905, 'eval_samples_per_second': 115.069, 'eval_steps_per_second': 14.384, 'epoch': 4.0}
{'loss': 0.0245, 'grad_norm': 0.046411607414484024, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.028193579986691475, 'eval_runtime': 8.6024, 'eval_samples_per_second': 116.246, 'eval_steps_per_second': 14.531, 'epoch': 5.0}
{'train_runtime': 613.3616, 'train_samples_per_second': 32.607, 'train_steps_per_second': 4.076, 'train_loss': 0.03874757385253906, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=0.03874757385253906, metrics={'train_runtime': 613.3616, 'train_samples_per_second': 32.607, 'train_steps_per_second': 4.076, 'total_flos': 5268505006080000.0, 'train_loss': 0.03874757385253906, 'epoch': 5.0})

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
import numpy as np

class InferenceDataset(Dataset):
    """A simple Dataset that wraps a list of raw strings."""
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # tokenize on the fly
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
            padding="max_length"
        )
        # squeeze out the batch dim
        return {k: v.squeeze(0) for k, v in encoding.items()}


def collate_fn(batch):
    # Stack dicts of tensors into a single dict of batched tensors
    return {k: torch.stack([d[k] for d in batch]) for k in batch[0]}

inference_dataset = InferenceDataset(eval_texts, tokenizer)
dataloader = DataLoader(
        inference_dataset,
        batch_size=8,          # tune this to fill your GPU without OOM
        collate_fn=collate_fn,  # so varying lengths get padded properly
        num_workers=4,          # or more, if you have CPU cores to spare
        pin_memory=True         # helps when using a GPU
    )
all_preds = []
with torch.no_grad():
    for batch in dataloader:
        # move to device
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = title_model(**batch)
        preds = outputs.logits.sigmod()
        print("REACHED HERE")
        all_preds.extend(preds)
probability = all_preds
from sklearn.metrics import f1_score,  recall_score, precision_score, multilabel_confusion_matrix
threshold = 0.5
targets = eval_labels
outputs = np.array(probability) >= threshold
f1_micro = f1_score(targets, outputs, average='micro')
f1_macro = f1_score(targets, outputs, average='macro')
recall = recall_score(targets,outputs)
precision = precision_score(targets, outputs)
multilabel_confusion_matrix(targets, outputs)

print(multilabel_confusion_matrix)

print("f1_micro", f1_macro )

print("f1_macro", f1_macro)

print("Recall", recall)
print("Precision", precision)

## Approach 1b: Using both Title and Summaries

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

train_texts = train_df['title_and_summary'].tolist()
train_labels = labels_list_train

eval_texts = test_df['title_and_summary'].tolist()
eval_labels = labels_list_test

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_encodings = tokenizer(train_texts, padding="max_length", truncation=True, max_length=512)
eval_encodings = tokenizer(eval_texts, padding="max_length", truncation=True, max_length=512)


class TextClassifierDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float).to(device)
        return item

train_dataset = TextClassifierDataset(train_encodings, train_labels)
eval_dataset = TextClassifierDataset(eval_encodings, eval_labels)

title_summary_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=len(label_columns),
)

title_summary_model = title_summary_model.to(device)

training_arguments = TrainingArguments(
    output_dir=".",
    dataloader_pin_memory=False,
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
)

trainer = Trainer(
    model=title_summary_model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2500 [00:00<?, ?it/s]

{'loss': 0.0925, 'grad_norm': 0.09874846786260605, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.03378874436020851, 'eval_runtime': 8.5884, 'eval_samples_per_second': 116.436, 'eval_steps_per_second': 14.555, 'epoch': 1.0}
{'loss': 0.0316, 'grad_norm': 0.17870688438415527, 'learning_rate': 3e-05, 'epoch': 2.0}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.024166790768504143, 'eval_runtime': 8.8188, 'eval_samples_per_second': 113.394, 'eval_steps_per_second': 14.174, 'epoch': 4.0}
{'loss': 0.0247, 'grad_norm': 0.06492176651954651, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.023755107074975967, 'eval_runtime': 8.666, 'eval_samples_per_second': 115.394, 'eval_steps_per_second': 14.424, 'epoch': 5.0}
{'train_runtime': 621.4687, 'train_samples_per_second': 32.182, 'train_steps_per_second': 4.023, 'train_loss': 0.0403738956451416, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=0.0403738956451416, metrics={'train_runtime': 621.4687, 'train_samples_per_second': 32.182, 'train_steps_per_second': 4.023, 'total_flos': 5268505006080000.0, 'train_loss': 0.0403738956451416, 'epoch': 5.0})

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
import numpy as np

class InferenceDataset(Dataset):
    """A simple Dataset that wraps a list of raw strings."""
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # tokenize on the fly
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
            padding="max_length"
        )
        # squeeze out the batch dim
        return {k: v.squeeze(0) for k, v in encoding.items()}


def collate_fn(batch):
    # Stack dicts of tensors into a single dict of batched tensors
    return {k: torch.stack([d[k] for d in batch]) for k in batch[0]}

inference_dataset = InferenceDataset(eval_texts, tokenizer)
dataloader = DataLoader(
        inference_dataset,
        batch_size=8,          # tune this to fill your GPU without OOM
        collate_fn=collate_fn,  # so varying lengths get padded properly
        num_workers=4,          # or more, if you have CPU cores to spare
        pin_memory=True         # helps when using a GPU
    )
all_preds = []
with torch.no_grad():
    for batch in dataloader:
        # move to device
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = title_model(**batch)
        preds = outputs.logits.sigmod()
        all_preds.extend(preds)
probability = all_preds
from sklearn.metrics import f1_score,  recall_score, precision_score, multilabel_confusion_matrix
threshold = 0.5
targets = eval_labels
outputs = np.array(probability) >= threshold
f1_micro = f1_score(targets, outputs, average='micro')
f1_macro = f1_score(targets, outputs, average='macro')
recall = recall_score(targets,outputs)
precision = precision_score(targets, outputs)
multilabel_confusion_matrix(targets, outputs)

print(multilabel_confusion_matrix)

print("f1_micro", f1_macro )

print("f1_macro", f1_macro)

print("Recall", recall)
print("Precision", precision)

In [ ]:
# Inference

In [ ]:
from transformers.convert_graph_to_onnx import convert

model.save_pretrained("./bert_imdb")
convert(
    framework="pt",
    model="./bert_imdb",
    output="./bert_imdb.onnx",
    opset=12,
    tokenizer=tokenizer,
)

# Approach 2: Feed multilabel classification data from parent to child

# Approach 3: Have 2 models separate

# Approach 4: GNNs?